In [ ]:
pip install numpy matplotlib opencv-python tensorflow flask

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 53s 109ms/step - accuracy: 0.7603 - loss: 0.7361 - val_accuracy: 0.9763 - val_loss: 0.0710
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 82s 109ms/step - accuracy: 0.9591 - loss: 0.1355 - val_accuracy: 0.9860 - val_loss: 0.0452
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 50s 108ms/step - accuracy: 0.9712 - loss: 0.0978 - val_accuracy: 0.9868 - val_loss: 0.0388
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 84s 113ms/step - accuracy: 0.9754 - loss: 0.0829 - val_accuracy: 0.9884 - val_loss: 0.0345
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 79s 107ms/step - accuracy: 0.9801 - loss: 0.0686 - val_accuracy: 0.9896 - val_loss: 0.0295
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 51s 108ms/step - accuracy: 0.9819 - loss: 0.0604 - val_accuracy: 0.9906 - val_loss: 0.0272
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 50s 107ms/step - accuracy: 0.9827 - loss: 0.0552 - val_accuracy: 0.9916 - val_loss: 0.0237
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 50s 107ms/step - accuracy: 0.9840 - loss: 0

In [ ]:
model.save("mnist-model.h5")





In [ ]:
from google.colab import files
uploaded = files.upload()


Saving data.zip to data.zip


In [ ]:
import zipfile
import os

zip_ref = zipfile.ZipFile("data.zip", "r")
zip_ref.extractall("data")
zip_ref.close()


In [ ]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
import zipfile


with zipfile.ZipFile("data.zip", "r") as zip_ref:
    zip_ref.extractall("data")


from google.colab import files
uploaded = files.upload()

model = load_model("mnist-model.h5")


def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = cv2.bitwise_not(img)
    img = img.astype("float32") / 255.0
    img = img.reshape(1, 28, 28, 1)
    return img

# 📁
main_folder = "/content/data"

for digit_folder in os.listdir(main_folder):
    folder_path = os.path.join(main_folder, digit_folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.lower().endswith((".jpg", ".png")):
                image_path = os.path.join(folder_path, filename)
                img = preprocess_image(image_path)
                prediction = model.predict(img)
                predicted_digit = prediction.argmax()
                confidence = prediction.max()
                print(f"Image: {filename} (Label: {digit_folder}) => Predicted: {predicted_digit} | Confidence: {confidence:.2f}")

Saving mnist.npz to mnist.npz


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

# ✅ Confirm folders are correct
print("Folders in /content/data/data:", os.listdir("/content/data/data"))

# ✅ Create image generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.1
)


# ✅ Load training data
train_data = train_datagen.flow_from_directory(
    '/content/data/data',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    subset='training',
    shuffle=True
)

# ✅ Load validation data
val_data = train_datagen.flow_from_directory(
    '/content/data/data',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    subset='validation',
    shuffle=True
)

# ✅ Define the CNN model
model = Sequential([
    tf.keras.Input(shape=(28, 28, 1)),  # <-- Use Input() instead of input_shape in Conv2D
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# ✅ Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ✅ Train the model
model.fit(train_data, validation_data=val_data, epochs=20)


# ✅ Save the model
model.save("custom_digit_model.h5")


Folders in /content/data/data: ['4', '1', '8', '6', '2', '3', '9', '5', '7', '0']
Found 773 images belonging to 10 classes.
Found 187 images belonging to 10 classes.
Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.1105 - loss: 2.3188 - val_accuracy: 0.1979 - val_loss: 2.2778
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.1661 - loss: 2.2715 - val_accuracy: 0.2513 - val_loss: 2.2362
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.1998 - loss: 2.2265 - val_accuracy: 0.2834 - val_loss: 2.1439
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.2438 - loss: 2.1322 - val_accuracy: 0.3316 - val_loss: 2.0221
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.3454 - loss: 2.0003 - val_accuracy: 0.4064 - val_loss: 1.8388
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.3675 - loss: 1.7984 - val_accuracy: 0.4599 - val_loss: 1.6115
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.4027 - 

In [ ]:
import os

print("Folders in /content/data/data :")
print(os.listdir("/content/data/data"))


Folders in /content/data/data :
['4', '1', '8', '6', '2', '3', '9', '5', '7', '0']


In [ ]:
!pip install flask pyngrok opencv-python
!python app.py


2025-06-01 13:17:32.538519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748783852.575521   21341 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748783852.587195   21341 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-01 13:17:37.934300: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Your app is live at: NgrokTunnel: "https://c5fd-34-106-193-9.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: off
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a prod

In [ ]:
def preprocess_image(path):
    import cv2
    import numpy as np

    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.GaussianBlur(img, (5, 5), 0)

    # Adaptive threshold handles lighting better
    img = cv2.adaptiveThreshold(img, 255,
                                 cv2.ADAPTIVE_THRESH_MEAN_C,
                                 cv2.THRESH_BINARY_INV, 11, 2)

    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        digit = img[y:y+h, x:x+w]
    else:
        digit = img

    digit = cv2.resize(digit, (20, 20))
    digit = np.pad(digit, ((4, 4), (4, 4)), mode='constant', constant_values=0)

    digit = digit.astype("float32") / 255.0
    digit = digit.reshape(1, 28, 28, 1)
    return digit


In [ ]:
!pip install flask flask-ngrok


In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

app = Flask(__name__)
run_with_ngrok(app)  # Automatically opens ngrok tunnel

model = load_model("custom_digit_model.h5")
UPLOAD_FOLDER = "static"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

def preprocess_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = cv2.bitwise_not(img)
    img = img.astype("float32") / 255.0
    img = img.reshape(1, 28, 28, 1)
    return img

@app.route("/", methods=["GET", "POST"])
def index():
    digit = None
    confidence = None
    if request.method == "POST":
        file = request.files["image"]
        path = os.path.join(app.config["UPLOAD_FOLDER"], "result.png")
        file.save(path)
        img = preprocess_image(path)
        prediction = model.predict(img)
        digit = int(np.argmax(prediction))
        confidence = round(float(np.max(prediction)) * 100, 2)
    return render_template("index.html", digit=digit, confidence=confidence)

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-40:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           

In [ ]:
!python app.py

2025-06-01 13:29:35.396479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748784575.433703   24638 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748784575.444895   24638 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-01 13:29:40.667377: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Your app is live at: NgrokTunnel: "https://bbbd-34-106-193-9.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: off
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a prod

In [ ]:
!pip install pyngrok


In [ ]:
!ngrok config add-authtoken 2xsByqyvwCXsfjeMyr2prc0gwsY_5t9Vam2ojjsf3PRFxfy6h


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!python app.py


2025-06-01 13:30:36.231710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748784636.269980   24916 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748784636.281113   24916 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-01 13:30:41.153106: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Your app is live at: NgrokTunnel: "https://5d77-34-106-193-9.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: off
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a prod

In [ ]:
!pip install pyngrok
